In [39]:
import pandas as pd
import numpy as np

from pandas.io import data, wb
import datetime
import os.path
import math


In [40]:
df_train = pd.read_csv('data/spy-ohlc.csv', index_col='Date', parse_dates=True)
df_dividends = pd.read_csv('data/spy-dividends.csv', index_col='Date', parse_dates=True)
df_train = df_train.join(df_dividends)
df_train['dividend'].fillna(0, inplace=True)

In [41]:
pma10 = df_train['Close'].rolling(window=10,center=False).mean() + df_train['dividend'].rolling(window=10, center=False).mean()
pma20 = df_train['Close'].rolling(window=20,center=False).mean()+ df_train['dividend'].rolling(window=20, center=False).mean()
pma50 = df_train['Close'].rolling(window=50,center=False).mean()+ df_train['dividend'].rolling(window=50, center=False).mean()
pma200 = df_train['Close'].rolling(window=200,center=False).mean()+ df_train['dividend'].rolling(window=200, center=False).mean()
vma50 = df_train['Volume'].rolling(window=50, center=False).mean()



In [42]:
#distance from MAs
diffma10  = (df_train['Close']/pma10 -1)*100
diffma20 = (df_train['Close']/pma20 -1)*100
diffma50 = (df_train['Close']/pma50 -1)*100
diffma200 = (df_train['Close']/pma200 -1)*100

#tangential of ma lines
pma10tan = (pma10[1:]/pma10[:-1].values-1)*100
pma20tan = (pma20[1:]/pma20[:-1].values-1)*100
pma50tan = (pma50[1:]/pma50[:-1].values-1)*100
pma200tan = (pma200[1:]/pma200[:-1].values-1)*100

#sequence of higher high/higher low
hhx1 = df_train['High'][1:] + df_train['dividend'][1:] > df_train['High'][:-1] 
hlx1 = df_train['Low'][1:] + df_train['dividend'][1:] > df_train['Low'][:-1] 
hhx2 = hhx1.shift(1)
hhx3 = hhx1.shift(2)
hlx2 = hlx1.shift(1)
hlx3 = hlx1.shift(2)

#rolling average of hh and hl
hh10 = hhx1.rolling(window=10, center=False).mean()
hl10 = hhx1.rolling(window=10, center=False).mean()

nvx1 = df_train['Volume']/vma50


In [43]:
#add features for training data

#distance from MAs
df_train = df_train.assign(diffma10=diffma10)
df_train = df_train.assign(diffma20=diffma20)
df_train = df_train.assign(diffma50=diffma50)
df_train = df_train.assign(diffma200=diffma200)

#tangential of ma lines
df_train = df_train.assign(pma10tan=pma10tan)
df_train = df_train.assign(pma20tan=pma20tan)
df_train = df_train.assign(pma50tan=pma50tan)
df_train = df_train.assign(pma200tan=pma200tan)

df_train = df_train.assign(hhx1=hhx1)
df_train = df_train.assign(hhx2=hhx2)
df_train = df_train.assign(hhx3=hhx3)

df_train = df_train.assign(hlx1=hlx1)
df_train = df_train.assign(hlx2=hlx2)
df_train = df_train.assign(hlx3=hlx3)

df_train = df_train.assign(hh10=hh10)
df_train = df_train.assign(hl10=hl10)

#last day volume
df_train = df_train.assign(nvx1=nvx1)

In [44]:
df_train.to_csv("data/spy-train.csv")
remove = ['Open','High','Low','Close','Volume','Adj Close','dividend']
df_train = df_train.drop(remove, axis=1)

In [45]:
df_train.describe()

,diffma10,diffma20,diffma50,diffma200,pma10tan,pma20tan,pma50tan,pma200tan,hh10,hl10,nvx1
count,5825.000000,5815.000000,5785.000000,5635.000000,5824.000000,5814.000000,5784.000000,5634.000000,5824.000000,5824.000000,5785.000000
mean,0.116030,0.256415,0.662056,2.818822,0.026637,0.026396,0.025582,0.026579,0.536075,0.536075,1.040060
std,1.775610,2.505772,3.929675,8.415483,0.322917,0.224390,0.138288,0.076736,0.162091,0.162091,0.529418
min,-15.974173,-21.374952,-27.180044,-39.537778,-2.996952,-1.671173,-0.983992,-0.346558,0.000000,0.000000,0.015866
25%,-0.752111,-0.948978,-1.173019,-1.027956,-0.130952,-0.086768,-0.033540,-0.002142,0.400000,0.400000,0.739572
50%,0.302137,0.577251,1.140054,4.101910,0.053302,0.052864,0.040958,0.041414,0.500000,0.500000,0.944072
75%,1.144593,1.763555,3.132791,8.490068,0.208046,0.160401,0.109845,0.075769,0.600000,0.600000,1.224154
max,8.571041,10.362980,14.114149,20.452184,1.869639,0.986241,0.549137,0.222489,1.000000,1.000000,12.385631


In [46]:
def predict(X_train, y_train, X_test):  
    from sklearn.cross_validation import train_test_split
    from sklearn import cross_validation
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import SVC


    #model = LogisticRegression(random_state=1)
    
    #model = RandomForestClassifier(n_estimators=350)
    model = GradientBoostingClassifier(n_estimators=100)
    #best so far
    #model = KNeighborsClassifier(n_neighbors=15)
    
    
    from sklearn import metrics
    
    #y_eval_comp = y_eval_pred[:,1] > 0.55
    #numpy.count_nonzero(y_eval_comp)
    
    #print("recall:", metrics.recall_score(y_eval, y_eval_comp, average='binary'))
    #print("precision:", metrics.precision_score(y_eval, y_eval_comp, average='binary'))
    #print("f1 score:", metrics.f1_score(y_eval, y_eval_comp, average='binary'))
    
    scores = cross_validation.cross_val_score(model, X_train, y_train, cv=5, scoring='precision')
    print("Precision score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))
    
    model.fit(X_train, y_train)
    y_test_pred = model.predict_proba(X_test);
    print(y_test_pred)
    return



In [47]:
X_train = df_train.values[200:-10, :]
X_test = df_train.values[-10:,:]
print(len(X_train))



5624


In [48]:
#for hh, shift one position back to get future value for hh
#print(hh.shift(-1).tail())
print(len(hhx1), len(hhx1.shift(-1)))
y_train = df_train['hhx1'].shift(-1).values[200:-10].astype(bool)
print(len(y_train))
print("predict hh")
predict(X_train, y_train, X_test)

#for hl
print("predict hl")
y_train = df_train['hlx1'].shift(-1).values[200:-10].astype(bool)
predict(X_train, y_train, X_test)

5833 5833
5624
predict hh
Precision score: 0.60 (+/- 0.01)
[[ 0.41313654  0.58686346]
 [ 0.36206505  0.63793495]
 [ 0.47970183  0.52029817]
 [ 0.48741931  0.51258069]
 [ 0.49147902  0.50852098]
 [ 0.55987391  0.44012609]
 [ 0.43308458  0.56691542]
 [ 0.35250935  0.64749065]
 [ 0.48462682  0.51537318]
 [ 0.400317    0.599683  ]]
predict hl
Precision score: 0.61 (+/- 0.02)
[[ 0.39430541  0.60569459]
 [ 0.34052299  0.65947701]
 [ 0.47874628  0.52125372]
 [ 0.51053832  0.48946168]
 [ 0.51909333  0.48090667]
 [ 0.50930945  0.49069055]
 [ 0.46160447  0.53839553]
 [ 0.42602295  0.57397705]
 [ 0.50878395  0.49121605]
 [ 0.50011669  0.49988331]]


In [49]:
def e_predict(X_train, y_train, X_test):
    from sklearn import cross_validation
    from sklearn.linear_model import LogisticRegression
    from sklearn.naive_bayes import GaussianNB
    from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.ensemble import VotingClassifier

    clf1 = LogisticRegression(random_state=1)
    clf2 = RandomForestClassifier(n_estimators=350)
    clf3 = GaussianNB()
    clf4 = GradientBoostingClassifier(n_estimators=100)
    clf5 = KNeighborsClassifier(n_neighbors=15)
    
    eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3), 
                                        ('gb', clf4), ('knn',clf5)], voting='soft')

    for clf in [clf1, clf2, clf3, clf4, clf5, eclf]: 
        print(type(clf).__name__, "\n")
        scores = cross_validation.cross_val_score(clf, X_train, y_train, cv=5, scoring='precision')
        print("Score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))
        clf.fit(X_train, y_train)
        y_test_pred = clf.predict_proba(X_test);
        print(y_test_pred)
